In [1]:
import numpy as np
u = np.array([6, 0, 3, 6]).reshape(-1, 1)
v = np.array([4, 2, 1]).reshape(-1, 1)
x = u@v.T
x.tolist()

[[24, 12, 6], [0, 0, 0], [12, 6, 3], [24, 12, 6]]

In [2]:
y = np.array([5, 0, 7, 0, 2, 0, 4, 0, 0, 0, 3, 6]).reshape(4, 3)
res = np.where(y != 0, (x-y)**2, 0)
print(np.sum(res))
j = 0.5*(np.sum(res) + np.sum(u**2) + np.sum(v**2))
print(j)

511
306.5


In [3]:
Y = np.array([5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1]).reshape(4, 7)
U = np.array([1, 5, 5, 1, 5, 5, 1, 1]).reshape(4, 2)
Vt = np.linalg.inv(U.T@U)@U.T@Y
print(np.round(Vt.tolist(), 3))

[[-0.  1. -0.  1. -0.  1. -0.]
 [ 1.  0.  1.  0.  1.  0.  1.]]


In [4]:
B0 = np.array([[1, 10], [1, 10], [10, 1], [1, 10], [10, 1] ])
Z0 = np.array([[1], [1], [5], [1], [5]])
ua = np.linalg.inv(B0.T@B0 + np.identity(2))@B0.T@Z0
np.round(ua, 3)

array([[0.492],
       [0.051]])

In [5]:
u_a_no_b = np.array( [[0.50148126], [0.0562376 ]])
u_a, b_u_a = np.array([[ 0.22024566], [-0.22193986]]), 0.007623888182962335

In [6]:
# no offset
u_a_no_b.reshape(1, -1)@np.array([10, 1]).reshape(-1, 1)

array([[5.0710502]])

In [7]:
u_a.reshape(1, -1)@np.array([10, 1]).reshape(-1, 1) + b_u_a

array([[1.98814063]])

In [8]:
u_a_no_b.reshape(1, -1)@np.array([1, 10]).reshape(-1, 1)

array([[1.06385726]])

In [9]:
u_a.reshape(1, -1)@np.array([1, 10]).reshape(-1, 1) + b_u_a + 5

array([[3.00847095]])

In [10]:
from code_for_hw12 import *

With offsets [[ 0.22024566]
 [-0.22193986]] 0.007623888182962335
With no offsets [[0.50148126]
 [0.0562376 ]]
ALS result for k = 2 : rmse train = [[0.00524272]] ; rmse validate = [[0.00524272]]
SGD result for k = 2 : rmse train = [[0.00462028]] ; rmse validate = [[0.00462028]]


In [11]:
data = load_ratings_data()
movies_dict, genres_dict = load_movies()
model = mf_als(data, None, k=10, lam=1, max_iter=20)
save_model(model)

Loading from ratings.csv users 13366 items 2000
Iteration 1 finished. Total Elapsed Time: 3.12
Iteration 2 finished. Total Elapsed Time: 6.15
Iteration 3 finished. Total Elapsed Time: 9.23
Iteration 4 finished. Total Elapsed Time: 12.25
Iteration 5 finished. Total Elapsed Time: 15.16
Iteration 6 finished. Total Elapsed Time: 18.18
Iteration 7 finished. Total Elapsed Time: 22.03
Iteration 8 finished. Total Elapsed Time: 25.65
Iteration 9 finished. Total Elapsed Time: 29.85
Iteration 10 finished. Total Elapsed Time: 35.02
Iteration 11 finished. Total Elapsed Time: 39.21
Iteration 12 finished. Total Elapsed Time: 43.96
Iteration 13 finished. Total Elapsed Time: 47.70
Iteration 14 finished. Total Elapsed Time: 51.28
Iteration 15 finished. Total Elapsed Time: 54.88
Iteration 16 finished. Total Elapsed Time: 58.56
Iteration 17 finished. Total Elapsed Time: 63.71
Iteration 18 finished. Total Elapsed Time: 69.83
Iteration 19 finished. Total Elapsed Time: 75.69
Iteration 20 finished. Total Elap

In [12]:
model = load_model()
(u, b_u, v, b_v) = model
user_id = 270894

In [13]:
genre_dict = {genre: 0 for genre in genres}
seen = []
for a, i, r in data:
    if a == user_id:
        seen.append(i)
    if a == user_id and r == 5:
        for genre in genres_dict[i]:
            genre_dict[genre] += 1
print(genre_dict)
print(max(genre_dict, key=genre_dict.get))

{'Western': 0, 'Comedy': 42, 'Children': 58, 'Crime': 6, 'Musical': 23, 'Adventure': 46, 'Drama': 27, 'Horror': 3, 'War': 2, 'Documentary': 0, 'Romance': 16, 'Animation': 74, 'Film-Noir': 0, 'Sci-Fi': 15, 'Mystery': 4, 'Fantasy': 31, 'IMAX': 12, 'Action': 20, 'Thriller': 8}
Animation


In [14]:
prediction = []
for key in movies_dict:
    if key not in seen:
        pred = u[user_id].T@v[key] + b_u[user_id] + b_v[key]
        #print(pred)
        prediction.append((key, genres_dict[key], pred.item()))
sorted_pred = sorted(prediction, reverse=True, key=lambda x: x[2])

n = 0
for i in range(50):
    if 'Animation' in sorted_pred[i][1]:
        n += 1
print(n)

21


In [15]:
def similarity(ia, ib):
    # ia and ib are indices
    va, vb = v[ia], v[ib]
    if ia != ib:
        return (sum(va*vb)/(np.linalg.norm(va)*np.linalg.norm(vb))).item()
    else:
        return 0

In [16]:
ia = 260
sims = []
for key in movies_dict:
    sim = similarity(ia, key)
    sims.append((key, sim))
sims_sorted = sorted(sims,reverse=True, key=lambda x: x[1])

sim260 = []
n = 0

for key, r in sims_sorted:
    if key in movies_dict:
        print(movies_dict[key])
        sim260.append(key)
        n += 1
    if n == 10:
        break
print(sim260)

Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode VI - Return of the Jedi (1983)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Star Wars: Episode III - Revenge of the Sith (2005)
Indiana Jones and the Last Crusade (1989)
"Lord of the Rings: The Two Towers, The (2002)"
"Lord of the Rings: The Fellowship of the Ring, The (2001)"
Ben-Hur (1959)
"Lord of the Rings: The Return of the King, The (2003)"
"Lord of the Rings, The (1978)"
[1196, 1210, 1198, 33493, 1291, 5952, 4993, 1287, 7153, 2116]


In [17]:
ia = 2628
sims = []
for key in movies_dict:
    sim = similarity(ia, key)
    sims.append((key, sim))
sims_sorted = sorted(sims,reverse=True, key=lambda x: x[1])

sim2628 = []
n = 0

for key, r in sims_sorted:
    if key in movies_dict:
        print(movies_dict[key])
        sim2628.append(key)
        n += 1
    if n == 10:
        break
print(sim2628)

Star Wars: Episode II - Attack of the Clones (2002)
Star Wars: Episode III - Revenge of the Sith (2005)
Indiana Jones and the Kingdom of the Crystal Skull (2008)
"Matrix Reloaded, The (2003)"
Pirates of the Caribbean: At World's End (2007)
Pirates of the Caribbean: Dead Man's Chest (2006)
Spider-Man 3 (2007)
"Matrix Revolutions, The (2003)"
"Lord of the Rings, The (1978)"
"X-Files: Fight the Future, The (1998)"
[5378, 33493, 59615, 6365, 53125, 45722, 52722, 6934, 2116, 1909]


In [41]:
key_li = sorted(list(movies_dict.keys()))

In [50]:
def ave_sim(index_list):
    sims = []
    for i in range(len(index_list)):
        ia = index_list[i]
        for j in range(i+1, len(index_list)):
            ib = index_list[j]
            sim = similarity(ia, ib)
            sims.append(sim)
    return np.mean(sims)

In [52]:
ave_sim(key_li)

0.004618852621916971

In [48]:
indices_by_genres = {}
for key in genres_dict:
    for genre in genres_dict[key]:
        try:
            indices_by_genres[genre].append(key)
        except:
            indices_by_genres[genre] = [key]

In [53]:
for genre in indices_by_genres:
    indices = indices_by_genres[genre]
    print(genre, ave_sim(indices))    

Adventure 0.07569592961693077
Animation 0.32538401087252183
Children 0.2706343858484362
Comedy 0.06823034872162335
Fantasy 0.0971507959688138
Romance 0.07036949414949016
Drama 0.06328884087958996
Action 0.12106102353909677
Crime 0.06505605368371047
Thriller 0.0709801119312924
Horror 0.24023964920332577
Mystery 0.07629346341343105
Sci-Fi 0.13710664217463053
War 0.1176460873947331
Musical 0.26344736882275616
IMAX 0.2700704450105055
Documentary 0.34008200754409407
Western 0.1476724994949762
Film-Noir 0.44912771006269586


In [56]:
def sim_between_genres(g1, g2):
    # g1 g2 are indices
    sims = []
    for i in range(len(g1)):
        ia = g1[i]
        for j in range(len(g2)):
            ib = g2[j]
            sim = similarity(ia, ib)
            sims.append(sim)
    return np.mean(sims)

In [57]:
g1 = indices_by_genres['Comedy']
for genre in indices_by_genres:
    if genre != 'Comedy':
        g2 = indices_by_genres[genre]
        res = sim_between_genres(g1, g2)
        print(genre, res)        

Adventure 0.007669058971249872
Animation 0.07189868577882132
Children 0.07596942465023895
Fantasy 0.028210924581107327
Romance 0.03296878557421564
Drama -0.022653552155691942
Action -0.014491908207798655
Crime -0.01698772707983787
Thriller -0.037209479741505014
Horror -0.04543365008962901
Mystery -0.03685894351614624
Sci-Fi -0.02250920152920314
War -0.0574214843121002
Musical 0.06982629035334435
IMAX -0.02781834570112349
Documentary 0.03452409476163227
Western -0.006164776046044933
Film-Noir -0.04187309625828562
